# 03. STRUCTURED

In [21]:
%pip install pyspark


Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
    Running setup.py install for pyspark: started
    Running setup.py install for pyspark: finished with status 'done'


You should consider upgrading via the 'c:\Users\anton\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


# Notas sobre los resultados las tablas creadas

- No hay datos sobre el Lote 0 en la tabla *CineticosIPC_Centrifugacion.xlsx*



In [14]:
import pandas as pd
import pandasql as psql

In [15]:
path_cleansed = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\02_cleansed\\'
path_structured = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\03_structured\\'

path_cent_02 = path_cleansed + 'Centrifugacion.xlsx'
path_OF_02 = path_cleansed + 'OF_123456.xlsx'
path_centrifugadoras_02 = path_cleansed + 'centrifugadoras.xlsx'
path_centIPC_02 = path_cleansed + 'CineticosIPC_Centrifugacion.xlsx'

# Leer el archivo Excel y guardar el contenido en un DataFrame
df_cent_02 = pd.read_excel(path_cent_02)
df_OF_02 = pd.read_excel(path_OF_02)
df_centrifugadoras_02 = pd.read_excel(path_centrifugadoras_02)
df_centIPC_02 = pd.read_excel(path_centIPC_02)

In [29]:
# Pivotamos la tabla Centrifugacion usando 'Orden' como clave para reorganizar las operaciones y sus correspondientes valores de 'DATETIME'
df_pivoted_cent= df_cent_02.pivot_table(index=['Orden', 'EQUIPO'], columns='Operación', values='DATETIME', aggfunc='first').reset_index()
display(df_pivoted_cent)

Operación,Orden,EQUIPO,Centrifugació 1 fi,Centrifugació 1 ini,Centrifugació 2 fi,Centrifugació 2 ini
0,10005144,14246,2023-05-26 15:26:57,2023-05-26 13:45:41,2023-05-26 18:18:12,2023-05-26 16:56:30
1,10005176,14246,2023-07-07 15:06:24,2023-07-07 11:53:24,2023-07-07 17:58:02,2023-07-07 16:15:02
2,10005183,14246,2023-09-08 16:18:56,2023-09-08 12:48:53,2023-09-08 18:47:19,2023-09-08 17:11:03
3,10005223,17825,2023-11-16 11:46:55,2023-11-16 08:38:56,2023-11-16 14:05:13,2023-11-16 12:36:47
4,10005233,14246,2023-11-23 12:01:12,2023-11-23 08:29:16,2023-11-23 14:24:31,2023-11-23 12:48:13
...,...,...,...,...,...,...
242,200209984,14246,2024-07-29 10:32:08,2024-07-29 07:14:41,2024-07-29 13:02:55,2024-07-29 11:21:19
243,200209985,17825,2024-07-31 11:05:43,2024-07-31 07:26:37,2024-07-31 13:36:41,2024-07-31 11:56:32
244,200210971,17825,2024-07-24 10:29:42,2024-07-24 06:47:07,2024-07-24 13:01:55,2024-07-24 11:18:34
245,200211467,17825,2024-08-01 10:46:06,2024-08-01 07:03:58,2024-08-01 13:13:04,2024-08-01 11:33:41


In [16]:
# Gracias al LEFT JOIN sólo nos quedamos con los lotes que aparecen en la tabla 0F_123456 (no hay que tenerlos en cuenta)
consulta = """
    SELECT 
    Lote,
    EQUIPO AS ID_centrifuga,
    `Centrifugació 1 ini` AS Inicio_Centrifugacion_1,
    `Centrifugació 1 fi` AS Fin_Centrifugacion_1,
    `Centrifugació 2 ini` AS Inicio_Centrifugacion_2,
    `Centrifugació 2 fi` AS Fin_Centrifugacion_2


    FROM df_OF_02 OF 
    LEFT JOIN df_pivoted_cent cent
    ON OF.OF = cent.Orden
    """

# Ejecutamos la consulta y generamos un dataframe
df_OF_cent = psql.sqldf(consulta, locals())
display(df_OF_cent)

,Lote,ID_centrifuga,Inicio_Centrifugacion_1,Fin_Centrifugacion_1,Inicio_Centrifugacion_2,Fin_Centrifugacion_2
0,23019,17825.0,2023-03-23 07:53:57,2023-03-23 10:29:43,2023-03-23 12:13:20,2023-03-23 13:03:38
1,23020,14246.0,2023-03-23 08:00:49,2023-03-23 10:26:25,2023-03-23 12:11:04,2023-03-23 13:34:10
2,23021,17825.0,2023-03-24 08:31:15,2023-03-24 11:53:41,2023-03-24 13:13:11,2023-03-24 14:30:45
3,23022,12912.0,2023-03-24 08:11:53,2023-03-24 11:32:53,2023-03-24 13:09:42,2023-03-24 14:29:39
4,23023,17825.0,2023-03-30 09:29:25,2023-03-30 12:26:49,2023-03-30 13:32:39,2023-03-30 14:52:53
...,...,...,...,...,...,...
202,24108,14246.0,2024-07-08 08:09:55,2024-07-08 11:07:41,2024-07-08 11:49:51,2024-07-08 13:32:56
203,24106,12912.0,2024-07-03 07:43:09,2024-07-03 11:00:11,2024-07-03 11:49:07,2024-07-03 13:19:16
204,24107,14246.0,2024-07-03 07:50:38,2024-07-03 11:08:40,2024-07-03 11:59:14,2024-07-03 13:33:24
205,24109,12912.0,2024-07-05 07:43:06,2024-07-05 11:13:40,2024-07-05 12:01:21,2024-07-05 13:43:20


In [17]:
# Calculamos -por lote- la media, el valor mínimo y el máximo de cada medida de la tabla CineticosIPC_Centrifugacion 
# en cada intervalo de tiempo de ambas centrifugaciones
consulta = '''
    SELECT
        OFc.Lote,
        ID_centrifuga,

        -- Métricas centrifugación 1 --

        Inicio_Centrifugacion_1,
        Fin_Centrifugacion_1,

        AVG(cd1.velocidad_separación) AS avg_velocidad_sep_1,
        MIN(cd1.velocidad_separación) AS min_velocidad_sep_1,
        MAX(cd1.velocidad_separación) AS max_velocidad_sep_1,

        AVG(cd1.presion_agua) AS avg_presion_agua_1,
        MIN(cd1.presion_agua) AS min_presion_agua_1,
        MAX(cd1.presion_agua) AS max_presion_agua_1,
        
        AVG(cd1.contrapresion) AS avg_contrapresion_1,
        MIN(cd1.contrapresion) AS min_contrapresion_1,
        MAX(cd1.contrapresion) AS max_contrapresion_1,

        AVG(cd1.caudal) AS avg_caudal_1,
        MIN(cd1.caudal) AS min_caudal_1,
        MAX(cd1.caudal) AS max_caudal_1,

        AVG(cd1.apertura_valvula_agua) AS avg_apertura_valvula_agua_1,
        MIN(cd1.apertura_valvula_agua) AS min_apertura_valvula_agua_1,
        MAX(cd1.apertura_valvula_agua) AS max_apertura_valvula_agua_1,

        AVG(cd1.descargas_totales) AS avg_descargas_totales_1,
        MIN(cd1.descargas_totales) AS min_descargas_totales_1,
        MAX(cd1.descargas_totales) AS max_descargas_totales_1,

        AVG(cd1.`descargas parciales`) AS avg_descargas_parciales_1,
        MIN(cd1.`descargas parciales`) AS min_descargas_parciales_1,
        MAX(cd1.`descargas parciales`) AS max_descargas_parciales_1,

        -- Métricas centrifugación 2 --

        Inicio_Centrifugacion_2,
        Fin_Centrifugacion_2,

        AVG(cd2.velocidad_separación) AS avg_velocidad_sep_2,
        MIN(cd2.velocidad_separación) AS min_velocidad_sep_2,
        MAX(cd2.velocidad_separación) AS max_velocidad_sep_2,

        AVG(cd2.presion_agua) AS avg_presion_agua_2,
        MIN(cd2.presion_agua) AS min_presion_agua_2,
        MAX(cd2.presion_agua) AS max_presion_agua_2,
        
        AVG(cd2.contrapresion) AS avg_contrapresion_2,
        MIN(cd2.contrapresion) AS min_contrapresion_2,
        MAX(cd2.contrapresion) AS max_contrapresion_2,

        AVG(cd2.caudal) AS avg_caudal_2,
        MIN(cd2.caudal) AS min_caudal_2,
        MAX(cd2.caudal) AS max_caudal_2,

        AVG(cd2.apertura_valvula_agua) AS avg_apertura_valvula_agua_2,
        MIN(cd2.apertura_valvula_agua) AS min_apertura_valvula_agua_2,
        MAX(cd2.apertura_valvula_agua) AS max_apertura_valvula_agua_2,

        AVG(cd2.descargas_totales) AS avg_descargas_totales_2,
        MIN(cd2.descargas_totales) AS min_descargas_totales_2,
        MAX(cd2.descargas_totales) AS max_descargas_totales_2,

        AVG(cd2.`descargas parciales`) AS avg_descargas_parciales_2,
        MIN(cd2.`descargas parciales`) AS min_descargas_parciales_2,
        MAX(cd2.`descargas parciales`) AS max_descargas_parciales_2

    FROM
        df_OF_cent OFc

    LEFT JOIN
        df_centrifugadoras_02 cd1
    ON
        OFc.ID_centrifuga = cd1.EQUIPO
        AND cd1.DateTime BETWEEN OFc.Inicio_Centrifugacion_1 AND OFc.Fin_Centrifugacion_1

    LEFT JOIN
        df_centrifugadoras_02 cd2
    ON
        OFc.ID_centrifuga = cd2.EQUIPO
        AND cd2.DateTime BETWEEN OFc.Inicio_Centrifugacion_2 AND OFc.Fin_Centrifugacion_2

    GROUP BY
        OFc.Lote;
'''

# Ejecutamos la consulta y generamos un dataframe
df_metrics_cent = psql.sqldf(consulta, locals())
display(df_metrics_cent)


,Lote,ID_centrifuga,Inicio_Centrifugacion_1,Fin_Centrifugacion_1,avg_velocidad_sep_1,min_velocidad_sep_1,max_velocidad_sep_1,avg_presion_agua_1,min_presion_agua_1,max_presion_agua_1,...,max_caudal_2,avg_apertura_valvula_agua_2,min_apertura_valvula_agua_2,max_apertura_valvula_agua_2,avg_descargas_totales_2,min_descargas_totales_2,max_descargas_totales_2,avg_descargas_parciales_2,min_descargas_parciales_2,max_descargas_parciales_2
0,0,14246.0,2023-07-07 11:53:24,2023-07-07 15:06:24,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,260.666834,1.192823,-0.004933,3.712791,NaN,NaN,NaN,3336.449750,0.000000,6675.081718
1,23019,17825.0,2023-03-23 07:53:57,2023-03-23 10:29:43,6679.593514,6678.278679,6680.694838,NaN,NaN,NaN,...,255.317259,26.471614,26.175997,26.732428,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,23020,14246.0,2023-03-23 08:00:49,2023-03-23 10:26:25,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,258.705689,2.678881,0.016039,3.545359,NaN,NaN,NaN,6650.426390,6518.431966,6677.551716
3,23021,17825.0,2023-03-24 08:31:15,2023-03-24 11:53:41,6680.760697,6678.192329,6683.191550,NaN,NaN,NaN,...,477.474482,26.528782,26.088005,27.419607,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,23022,12912.0,2023-03-24 08:11:53,2023-03-24 11:32:53,0.000000,0.000000,0.000000,0.357143,0.0,1.0,...,563.939978,2.600954,0.555305,3.259559,NaN,NaN,NaN,6678.741666,6677.179237,6681.402561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,24106,12912.0,2024-07-03 07:43:09,2024-07-03 11:00:11,0.000000,0.000000,0.000000,0.071429,0.0,1.0,...,255.335524,1.112902,-0.005150,3.329689,2.015265,1.198738,3.123587,3569.122378,0.000000,6664.209955
203,24107,14246.0,2024-07-03 07:50:38,2024-07-03 11:08:40,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,340.394400,1.679032,0.007609,3.511598,1.201718,0.107519,1.715353,4687.211868,0.000000,6682.231023
204,24108,14246.0,2024-07-08 08:09:55,2024-07-08 11:07:41,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,263.593067,1.334550,0.008328,3.210500,1.605238,1.442146,1.733981,3950.935798,0.000000,6677.068196
205,24109,12912.0,2024-07-05 07:43:06,2024-07-05 11:13:40,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,257.349788,1.026492,-0.005605,3.077578,2.049252,1.408552,3.090590,4080.770998,0.000000,6666.254359


In [18]:
# Eliminamos los lotes de CineticosIPC_Centrifugacion que no se encuentrene en OF_123456 (no hay que tenerlos en cuenta)
consulta = '''
    SELECT
        OF.Lote,
        Centrífuga,
        `Centrifugada (1 o 2)`,
        `Volumen centrifugado (L)`,
        Turbidez

    FROM
        df_OF_02 OF

    LEFT JOIN 
        df_centIPC_02 cIPC
    ON 
        OF.Lote = cIPC.Lote

'''

# Ejecutamos la consulta y generamos un dataframe
df_OF_centIPC = psql.sqldf(consulta, locals())
display(df_OF_centIPC)

,Lote,Centrífuga,Centrifugada (1 o 2),Volumen centrifugado (L),Turbidez
0,23019,17825.0,1.0,200.0,26.80
1,23019,17825.0,1.0,400.0,26.32
2,23019,17825.0,1.0,600.0,28.16
3,23019,17825.0,1.0,800.0,28.08
4,23019,17825.0,1.0,1000.0,28.40
...,...,...,...,...,...
2205,24110,14246.0,1.0,200.0,35.04
2206,24110,14246.0,1.0,1000.0,34.24
2207,24110,14246.0,1.0,2000.0,34.40
2208,24110,14246.0,2.0,100.0,28.00


In [30]:
# Pivotamos la tabla CineticosIPC_Centrifugacion (habiendo eliminado los lotes que no aparecen en OF_123456) 
# usando Lote y Centrífuga como claves para reorganizar las operaciones y sus correspondientes valores de Turbidez 
df_pivoted_centIPC= df_OF_centIPC.pivot_table(index=['Lote', 'Centrífuga'], columns=['Centrifugada (1 o 2)', 'Volumen centrifugado (L)'], values='Turbidez', aggfunc='first').reset_index()

# Renombramos los nombres de las columnas
df_pivoted_centIPC.columns = ['Lote', 'Centrífuga', 'Turbidez_1_100', 'Turbidez_1_200','Turbidez_1_400','Turbidez_1_600','Turbidez_1_800','Turbidez_1_1000','Turbidez_1_1200','Turbidez_1_1400','Turbidez_1_1600','Turbidez_1_1800','Turbidez_1_2000','Turbidez_2_100','Turbidez_2_200','Turbidez_2_300','Turbidez_2_400',]
display(df_pivoted_centIPC)

,Lote,Centrífuga,Turbidez_1_100,Turbidez_1_200,Turbidez_1_400,Turbidez_1_600,Turbidez_1_800,Turbidez_1_1000,Turbidez_1_1200,Turbidez_1_1400,Turbidez_1_1600,Turbidez_1_1800,Turbidez_1_2000,Turbidez_2_100,Turbidez_2_200,Turbidez_2_300,Turbidez_2_400
0,23019,17825.0,NaN,26.80,26.32,28.16,28.08,28.40,27.68,28.56,28.48,28.88,28.16,20.80,20.80,21.60,21.52
1,23020,14246.0,NaN,29.84,28.64,30.88,28.56,27.60,27.76,28.32,27.60,28.48,26.88,12.80,13.04,12.88,12.88
2,23021,17825.0,NaN,28.64,30.40,28.72,28.72,29.20,28.64,29.12,29.52,29.12,28.96,18.48,20.48,17.84,20.24
3,23022,12912.0,NaN,26.32,26.40,26.24,26.40,26.24,26.08,26.32,26.96,26.88,26.56,10.08,10.00,11.04,9.44
4,23023,17825.0,NaN,26.56,26.00,26.24,26.80,27.04,26.24,26.96,26.96,26.56,26.88,21.04,20.96,20.80,20.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,24106,12912.0,NaN,32.40,NaN,NaN,NaN,31.44,NaN,NaN,NaN,NaN,32.64,26.80,NaN,NaN,26.00
202,24107,14246.0,NaN,28.56,NaN,NaN,NaN,28.00,NaN,NaN,NaN,NaN,28.32,22.72,NaN,NaN,23.84
203,24108,14246.0,NaN,29.36,NaN,NaN,NaN,30.00,NaN,NaN,NaN,NaN,29.92,15.44,NaN,NaN,15.36
204,24109,12912.0,NaN,31.84,NaN,NaN,NaN,31.36,NaN,NaN,NaN,NaN,31.84,29.36,NaN,NaN,28.00


In [31]:
# Juntamos CineticosIPC_Centrifugacion pivotada con el dataframe df_metrics_cent 
consulta = '''
    SELECT
        mc.Lote,
        ID_centrifuga,

        -- Centrifugacion 1 --
        Inicio_Centrifugacion_1,
        Fin_Centrifugacion_1,
        avg_velocidad_sep_1,
        min_velocidad_sep_1,
        max_velocidad_sep_1,
        avg_presion_agua_1,
        min_presion_agua_1,
        max_presion_agua_1,
        avg_contrapresion_1,
        min_contrapresion_1,
        max_contrapresion_1,
        avg_caudal_1,
        min_caudal_1,
        max_caudal_1,
        avg_apertura_valvula_agua_1,
        min_apertura_valvula_agua_1,
        max_apertura_valvula_agua_1,
        avg_descargas_totales_1,
        min_descargas_totales_1,
        max_descargas_totales_1,
        avg_descargas_parciales_1,
        min_descargas_parciales_1,
        max_descargas_parciales_1,
        Turbidez_1_100,
        Turbidez_1_200,
        Turbidez_1_400,
        Turbidez_1_600,
        Turbidez_1_800,
        Turbidez_1_1000,
        Turbidez_1_1200,
        Turbidez_1_1400,
        Turbidez_1_1600,
        Turbidez_1_1800,
        Turbidez_1_2000,

        -- Centrifugacion 2 --
        Inicio_Centrifugacion_2,
        Fin_Centrifugacion_2,
        avg_velocidad_sep_2,
        min_velocidad_sep_2,
        max_velocidad_sep_2,
        avg_presion_agua_2,
        min_presion_agua_2,
        max_presion_agua_2,
        avg_contrapresion_2,
        min_contrapresion_2,
        max_contrapresion_2,
        avg_caudal_2,
        min_caudal_2,
        max_caudal_2,
        avg_apertura_valvula_agua_2,
        min_apertura_valvula_agua_2,
        max_apertura_valvula_agua_2,
        avg_descargas_totales_2,
        min_descargas_totales_2,
        max_descargas_totales_2,
        avg_descargas_parciales_2,
        min_descargas_parciales_2,
        max_descargas_parciales_2,
        Turbidez_2_100,
        Turbidez_2_200,
        Turbidez_2_300,
        Turbidez_2_400


    FROM
        df_metrics_cent mc

    LEFT JOIN 
        df_pivoted_centIPC cIPC
    ON 
        mc.Lote = cIPC.Lote

'''
# Ejecutar la consulta
df_cent_03 = psql.sqldf(consulta, locals())
display(df_cent_03)

,Lote,ID_centrifuga,Inicio_Centrifugacion_1,Fin_Centrifugacion_1,avg_velocidad_sep_1,min_velocidad_sep_1,max_velocidad_sep_1,avg_presion_agua_1,min_presion_agua_1,max_presion_agua_1,...,avg_descargas_totales_2,min_descargas_totales_2,max_descargas_totales_2,avg_descargas_parciales_2,min_descargas_parciales_2,max_descargas_parciales_2,Turbidez_2_100,Turbidez_2_200,Turbidez_2_300,Turbidez_2_400
0,0,14246.0,2023-07-07 11:53:24,2023-07-07 15:06:24,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,NaN,NaN,NaN,3336.449750,0.000000,6675.081718,NaN,NaN,NaN,NaN
1,23019,17825.0,2023-03-23 07:53:57,2023-03-23 10:29:43,6679.593514,6678.278679,6680.694838,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.80,20.80,21.60,21.52
2,23020,14246.0,2023-03-23 08:00:49,2023-03-23 10:26:25,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,NaN,NaN,NaN,6650.426390,6518.431966,6677.551716,12.80,13.04,12.88,12.88
3,23021,17825.0,2023-03-24 08:31:15,2023-03-24 11:53:41,6680.760697,6678.192329,6683.191550,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.48,20.48,17.84,20.24
4,23022,12912.0,2023-03-24 08:11:53,2023-03-24 11:32:53,0.000000,0.000000,0.000000,0.357143,0.0,1.0,...,NaN,NaN,NaN,6678.741666,6677.179237,6681.402561,10.08,10.00,11.04,9.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,24106,12912.0,2024-07-03 07:43:09,2024-07-03 11:00:11,0.000000,0.000000,0.000000,0.071429,0.0,1.0,...,2.015265,1.198738,3.123587,3569.122378,0.000000,6664.209955,26.80,NaN,NaN,26.00
203,24107,14246.0,2024-07-03 07:50:38,2024-07-03 11:08:40,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,1.201718,0.107519,1.715353,4687.211868,0.000000,6682.231023,22.72,NaN,NaN,23.84
204,24108,14246.0,2024-07-08 08:09:55,2024-07-08 11:07:41,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,1.605238,1.442146,1.733981,3950.935798,0.000000,6677.068196,15.44,NaN,NaN,15.36
205,24109,12912.0,2024-07-05 07:43:06,2024-07-05 11:13:40,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,2.049252,1.408552,3.090590,4080.770998,0.000000,6666.254359,29.36,NaN,NaN,28.00


In [32]:
# Guardamos el resultado en un archivo Excel
df_cent_03.to_excel(f'{path_structured}centrifugacion_total.xlsx', index=False)